# Using the API to interact with a remote MISP instance

You can fetch a VM from here: https://www.circl.lu/misp-images/latest/, or connect to your dev instance.

This box needs to be run in order to connect to the MISP instance and run the subsequent commands.

In [ ]:
from pymisp import ExpandedPyMISP, MISPEvent, MISPAttribute

# The URL of the MISP instance to connect to
misp_url = 'http://127.0.0.1:8080'
# Can be found in the MISP web interface under 
# http://+MISP_URL+/users/view/me -> Authkey
misp_key = 'aJAmQQoBhVL5jqUDSucIkPrEYIbFyW0wwQnxyBfc'
# Should PyMISP verify the MISP certificate
misp_verifycert = False

# Getting the API key (automatically generated on the trainig VM)

In [ ]:
from pathlib import Path

api_file = Path('apikey')
if api_file.exists():
    misp_url = 'http://127.0.0.1'
    misp_verifycert = False
    with open(api_file) as f:
        misp_key = f.read().strip()
    print(misp_key)

# Initialize PyMISP

In [ ]:
misp = ExpandedPyMISP(misp_url, misp_key, misp_verifycert)

# Get the last events

There are multiple definition for *last* in MISP.

## Last *published* events

In [ ]:
response = misp.search(publish_timestamp='2d')

print (response)
events = []
for event in response:
    me = MISPEvent()
    me.load(event)
    events.append(me)
    
for e in events:
    print(e)

In [ ]:
events = misp.search(publish_timestamp=['120m', '100m'], pythonify=True)

for e in events:
    print(e)

## Last *updated* events

In [ ]:
from datetime import datetime

ts = int(datetime.now().timestamp())

response = misp.search(timestamp=ts-36000)

events = []
for event in response:
    me = MISPEvent()
    me.load(event)
    events.append(me)

for e in events:
    print(e)

## On an interval

In [ ]:
from datetime import datetime
ts = int(datetime.now().timestamp())

events = misp.search(timestamp=[ts-3600, ts], pythonify=True)

for e in events:
    print(e)

# Get the last attributes

## Last *published* attributes

In [ ]:
response = misp.search(controller='attributes', publish_timestamp='1d')

attributes = []
for attribute in response['Attribute']:
    ma = MISPAttribute()
    ma.from_dict(**attribute)
    attributes.append(ma)

for a in attributes:
    print(a.event_id, a)

In [ ]:
response = misp.search(controller='attributes', publish_timestamp=['2d', '1h'])

attributes = []
for attribute in response['Attribute']:
    ma = MISPAttribute()
    ma.from_dict(**attribute)
    attributes.append(ma)

for a in attributes:
    print(a)

## Last *updated* attributes

In [ ]:
ts = int(datetime.now().timestamp())

response = misp.search(controller='attributes', timestamp=ts - 36000)

attributes = []
for attribute in response['Attribute']:
    ma = MISPAttribute()
    ma.from_dict(**attribute)
    attributes.append(ma)

for a in attributes:
    print(a)

# Fast search at index event level

You have multiple ways to search for different values in MISP. Searching in the medadata of the events is very fast and if generally the recommended approach if your query returns lots of events.

In [ ]:
response = misp.search_index(eventinfo='Cobalt Strike')

events = []
for event in response:
    me = MISPEvent()
    me.from_dict(**event)
    events.append(me)

for e in events:
    print(e)

In [ ]:
print('No attributes are in the event', events[0].attributes)

In [ ]:
response = misp.search_index(tags='malware_classification:malware-category="Ransomware"')

events = []
for event in response:
    me = MISPEvent()
    me.from_dict(**event)
    events.append(me)

for e in events:
    print(e)

In [ ]:
response = misp.search_index(timestamp='1h')

events = []
for event in response:
    me = MISPEvent()
    me.from_dict(**event)
    events.append(me)

for e in events:
    print(e)

In [ ]:
events[0].id

In [ ]:
event = MISPEvent()
#event.load(misp.get(events[0].id))
print (misp.get(events[0].id))
#print(event.to_json())

# Search indicators

In [ ]:
complex_query = misp.build_complex_query(or_parameters=['59.157.4.2', 'hotfixmsupload.com', '8.8.8.8'])
events = misp.search(value=complex_query, pythonify=True)

for e in events:
    print(e)

# Sightings

In [ ]:
misp.sighting(value=e.attributes[1].value)

In [ ]:
misp.sighting_list(e.attributes[1].id)

# Admin Stuff

In [ ]:
misp.get_sharing_groups()

## User

In [ ]:
misp.get_users_list()

In [ ]:
misp.add_user('bar@foo.de', 1, 3)

In [ ]:
misp.get_organisations_list()

In [ ]:
misp.get_roles_list()

In [ ]:
misp.get_feeds_list()

In [ ]:
misp.cache_feeds_all()